# Exploratory Data Analysis

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import re

## Load the Data into Pandas dataframes

In [2]:
job_postings = pd.read_csv('raw_data/job_postings.csv')
companies = pd.read_csv('raw_data/company_details/companies.csv')
company_industries = pd.read_csv('raw_data/company_details/company_industries.csv')
company_specialities = pd.read_csv('raw_data/company_details/company_specialities.csv')
employee_counts = pd.read_csv('raw_data/company_details/employee_counts.csv')
benefits = pd.read_csv('raw_data/job_details/benefits.csv')
job_industries = pd.read_csv('raw_data/job_details/job_industries.csv')
job_skills = pd.read_csv('raw_data/job_details/job_skills.csv')

In [3]:
print(job_postings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15886 entries, 0 to 15885
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      15886 non-null  int64  
 1   company_id                  15520 non-null  float64
 2   title                       15886 non-null  object 
 3   description                 15885 non-null  object 
 4   max_salary                  5521 non-null   float64
 5   med_salary                  981 non-null    float64
 6   min_salary                  5521 non-null   float64
 7   pay_period                  6502 non-null   object 
 8   formatted_work_type         15886 non-null  object 
 9   location                    15886 non-null  object 
 10  applies                     8700 non-null   float64
 11  original_listed_time        15886 non-null  float64
 12  remote_allowed              2340 non-null   float64
 13  views                       131

## ER Diagram before enforcing constraints

```mermaid
erDiagram
    Company {
        int company_id PK
        string name
        string description
    }

    CompanyIndustry {
        int company_id FK
        string industry
    }
    Company |o..o| CompanyIndustry : "part of"

    CompanySpeciality {
        int company_id FK
        string speciality
    }
    Company |o..o{ CompanySpeciality : has

    EmployeeCount {
        int company_id FK
        int employee_count
        int follower_count
        float time_recorded
    }
    Company |o..o{ EmployeeCount : has
    
    JobPosting {
        int job_id PK
        int company_id FK
    }
    JobPosting }o..o| Company : by
    
    JobSkill {
        int job_id FK
        string skill_abr
    }
    JobPosting |o..o{ JobSkill : has
    
    JobIndustry {
        int job_id FK
        string industry_id
    }
    JobPosting |o..o{ JobIndustry : has
    
    Benefit {
        int job_id FK
        bool inferred
        string type
    }
    JobPosting ||--o{ Benefit : has
```

## Enforce constraints

3 steps for each table
1. Remove duplicate rows (it helps to identify the PKs in the table)
2. Enforce FK constraint (helps identify if its 0..1 or 1..1 relationship)
3. Identify strong entities not present in weak entities (helps us to identify if its 1..N or 0..N relationship)

### JobSkills
Remove entries from job_skills for job_ids that don't exist in job_postings

In [4]:
# enforcing primary keys for job_skills table
print(f"Number of rows in job_skills before removing duplicate entries - {len(job_skills)}")
print(f"Number of rows with duplicate data - {len([x for x in job_skills.duplicated() if x==True])}")
job_skills.drop_duplicates(inplace=True)
print(f"Number of rows in job_skills after removing duplicate entries - {len(job_skills)}")
# enforcing foreign key constraint for job_skills table
a = set(job_postings["job_id"])
b = set(job_skills["job_id"])
print(f"Number of rows in job_skills before enforcing foreign key constraint - {len(job_skills)}")
print("These job_ids don't correspond to any row in job_postings. Removing them ...")
print(job_skills.loc[job_skills["job_id"].isin(b - a)])
job_skills.drop(job_skills[job_skills["job_id"].isin(b - a)].index, inplace=True)
print(job_skills.loc[job_skills["job_id"].isin(b - a)])
print(f"Number of rows in job_skills after enforcing foreign key constraint - {len(job_skills)}")
print(f"Number of rows in job_postings that don't have any skill associated with it - {len(a - b)}")
# general comment - this can be a functional component - if someone removes a job posting, then a trigger can be set such that
# the corresponding skills for that job should also be removed from job_skills table

Number of rows in job_skills before removing duplicate entries - 27899
Number of rows with duplicate data - 0
Number of rows in job_skills after removing duplicate entries - 27899
Number of rows in job_skills before enforcing foreign key constraint - 27899
These job_ids don't correspond to any row in job_postings. Removing them ...
           job_id skill_abr
11453  3697350868        BD
11454  3697350868       ENG
11455  3697350868      SALE
11456  3697350870        BD
11457  3697350870       EDU
...           ...       ...
13675  3697361082       ENG
13676  3697361082      RSCH
13677  3697361088        BD
13678  3697361088        IT
13679  3697361088      SALE

[932 rows x 2 columns]
Empty DataFrame
Columns: [job_id, skill_abr]
Index: []
Number of rows in job_skills after enforcing foreign key constraint - 26967
Number of rows in job_postings that don't have any skill associated with it - 637


### JobIndustries
Remove entries from job_industries for job_ids that don't exist in job_postings

In [5]:
print(f"Number of rows in job_industries before removing duplicate entries - {len(job_industries)}")
print(f"Number of rows with duplicate data - {len([x for x in job_industries.duplicated() if x==True])}")
job_industries.drop_duplicates(inplace=True)
print(f"Number of rows in job_industries after removing duplicate entries - {len(job_industries)}")
a = set(job_postings["job_id"])
b = set(job_industries["job_id"])
print(f"Number of rows in job_industries before enforcing foreign key constraint - {len(job_industries)}")
print("These job_ids don't correspond to any row in job_postings. Removing them ...")
print(job_industries.loc[job_industries["job_id"].isin(b - a)])
job_industries.drop(job_industries[job_industries["job_id"].isin(b - a)].index, inplace=True)
print(job_industries.loc[job_industries["job_id"].isin(b - a)])
print(f"Number of rows in job_industries after enforcing foreign key constraint - {len(job_industries)}")
print(f"Number of rows in job_postings that don't have any industry associated with it - {len(a - b)}")

# on delete cascade or trigger when a job id is deleted from job postings then that id should be deleted from job industries as well.

Number of rows in job_industries before removing duplicate entries - 21993
Number of rows with duplicate data - 0
Number of rows in job_industries after removing duplicate entries - 21993
Number of rows in job_industries before enforcing foreign key constraint - 21993
These job_ids don't correspond to any row in job_postings. Removing them ...
           job_id  industry_id
8996   3697350868            4
8997   3697350868           96
8998   3697350868            6
8999   3697350870           69
9000   3697350870          132
...           ...          ...
11045  3697361082           96
11046  3697361082            6
11047  3697361088            4
11048  3697361088           96
11049  3697361088            6

[1040 rows x 2 columns]
Empty DataFrame
Columns: [job_id, industry_id]
Index: []
Number of rows in job_industries after enforcing foreign key constraint - 20953
Number of rows in job_postings that don't have any industry associated with it - 314


### Benefits
Remove entries from benefits for job_ids that don't exist in job_postings

In [6]:
print(f"Number of rows in benefits before removing duplicate entries - {len(benefits)}")
print(f"Number of rows with duplicate data - {len([x for x in benefits.duplicated() if x==True])}")
benefits.drop_duplicates(inplace=True)
print(f"Number of rows in job_industries after removing duplicate entries - {len(benefits)}")
a = set(job_postings["job_id"])
b = set(benefits["job_id"])
print(f"Number of rows in benefits before enforcing foreign key constraint - {len(benefits)}")
print("These job_ids don't correspond to any row in job_postings. Removing them ...")
print(benefits.loc[benefits["job_id"].isin(b - a)])
benefits.drop(benefits[benefits["job_id"].isin(b - a)].index, inplace=True)
print(benefits.loc[benefits["job_id"].isin(b - a)])
print(f"Number of rows in benefits after enforcing foreign key constraint - {len(benefits)}")
print(f"Number of rows in job_postings that don't have any benefit associated with it - {len(a - b)}")

Number of rows in benefits before removing duplicate entries - 13761
Number of rows with duplicate data - 0
Number of rows in job_industries after removing duplicate entries - 13761
Number of rows in benefits before enforcing foreign key constraint - 13761
These job_ids don't correspond to any row in job_postings. Removing them ...
Empty DataFrame
Columns: [job_id, inferred, type]
Index: []
Empty DataFrame
Columns: [job_id, inferred, type]
Index: []
Number of rows in benefits after enforcing foreign key constraint - 13761
Number of rows in job_postings that don't have any benefit associated with it - 10390


### Companies
Handle discrepancies in companies and job_postings

In [7]:
idx = companies.duplicated(subset=list(companies.columns.difference(['company_id'])))
print(f"There are {len(companies)} rows in companies before removing duplicates.")
print(f"There are {idx.sum()} duplicate rows in companies, where only the company_id is different, but all other columns are equal.")
print("Removing these duplicate rows from companies...")
companies.drop(companies[idx].index, inplace=True)
print(f"There are {len(companies)} rows in companies before removing duplicates.")

There are 6063 rows in companies before removing duplicates.
There are 33 duplicate rows in companies, where only the company_id is different, but all other columns are equal.
Removing these duplicate rows from companies...
There are 6030 rows in companies before removing duplicates.


In [8]:
job_postings["company_id"] = job_postings["company_id"].fillna(0).astype(np.int64)
a = set(job_postings["company_id"])
b = set(companies["company_id"])
print(f"Number of unique company_id in companies which doesn't exist in job_postings - {len(b - a)}")
print(f"Number of unique company_id in job_postings which doesn't exist in companies - {len(a - b)}")
print(f"Replacing company_id in job_postings where company_id doesn't exist to 0 (Null) ...")
job_postings.loc[job_postings["company_id"].isin((a - {0}) - b), "company_id"] = 0
a = set(job_postings["company_id"])
b = set(companies["company_id"])
print(f"Number of unique company_id in companies which doesn't exist in job_postings - {len(b - a)}")
print(f"Number of unique company_id in job_postings which doesn't exist in companies - {len(a - b)}") # here 1 o/p represents null
job_postings["company_id"].replace(0, np.nan, inplace=True)
# Find companies who hasn't posted any job posting yet - 50 - functional component

Number of unique company_id in companies which doesn't exist in job_postings - 50
Number of unique company_id in job_postings which doesn't exist in companies - 51
Replacing company_id in job_postings where company_id doesn't exist to 0 (Null) ...
Number of unique company_id in companies which doesn't exist in job_postings - 50
Number of unique company_id in job_postings which doesn't exist in companies - 1


### EmployeeCounts
Remove entries from employee_counts for company_ids that don't exist in companies

In [9]:
print(f"Number of rows in employee_counts before removing duplicate entries - {len(employee_counts)}")
print(f"Number of rows with duplicate data - {len([x for x in employee_counts.duplicated() if x==True])}")
employee_counts.drop_duplicates(inplace=True)
print(f"Number of rows in employee_counts after removing duplicate entries - {len(employee_counts)}")
a = set(companies["company_id"])
b = set(employee_counts["company_id"])
print(f"Number of rows in employee_counts before enforcing foreign key constraint - {len(employee_counts)}")
print("These company_ids don't correspond to any row in companies. Removing them ...")
print(employee_counts.loc[employee_counts["company_id"].isin(b - a)])
employee_counts.drop(employee_counts[employee_counts["company_id"].isin(b - a)].index, inplace=True)
print(f"Number of rows in employee_counts after enforcing foreign key constraint - {len(employee_counts)}")
print(f"Number of rows in companies whose company don't have any employee_counts associated with it - {len(a - b)}")

Number of rows in employee_counts before removing duplicate entries - 15907
Number of rows with duplicate data - 3356
Number of rows in employee_counts after removing duplicate entries - 12551
Number of rows in employee_counts before enforcing foreign key constraint - 12551
These company_ids don't correspond to any row in companies. Removing them ...
    company_id  employee_count  follower_count  time_recorded
0     81149246               6              91   1.692645e+09
1     10033339               3             187   1.692645e+09
2      6049228              20              82   1.692645e+09
3      2641066              45            2336   1.692645e+09
4     96649998               0               2   1.692645e+09
5     82684341               3             128   1.692645e+09
6     82296828               0              64   1.692645e+09
7     86746333              11             478   1.692645e+09
8       718651               5              22   1.692645e+09
9      4781041             

### CompanyIndustries
Remove entries from company_industries for company_ids that don't exist in companies

In [10]:
print(f"Number of rows in company_industries before removing duplicate entries - {len(company_industries)}")
print(f"Number of rows with duplicate data - {len([x for x in company_industries.duplicated() if x==True])}")
company_industries.drop_duplicates(inplace=True)
print(f"Number of rows in company_industries after removing duplicate entries - {len(company_industries)}")
a = set(companies["company_id"])
b = set(company_industries["company_id"])
print(f"Number of rows in company_industries before enforcing foreign key constraint - {len(company_industries)}")
print("These company_ids don't correspond to any row in companies. Removing them ...")
print(company_industries.loc[company_industries["company_id"].isin(b - a)])
company_industries.drop(company_industries[company_industries["company_id"].isin(b - a)].index, inplace=True)
print(f"Number of rows in company_industries after enforcing foreign key constraint - {len(company_industries)}")
print(f"Number of rows in companies whose company don't have any company_industries associated with it - {len(a - b)}")

Number of rows in company_industries before removing duplicate entries - 15880
Number of rows with duplicate data - 9877
Number of rows in company_industries after removing duplicate entries - 6003
Number of rows in company_industries before enforcing foreign key constraint - 6003
These company_ids don't correspond to any row in companies. Removing them ...
    company_id                               industry
0     81149246                       Higher Education
1     10033339      Information Technology & Services
2      6049228                             Accounting
3      2641066  Electrical & Electronic Manufacturing
4     96649998                Marketing & Advertising
5     82684341                 Hospital & Health Care
6     82296828      Information Technology & Services
7     86746333               Logistics & Supply Chain
8       718651                       Medical Practice
9      4781041                     Mental Health Care
10     7573454      Information Technology & S

### CompanySpecialities
Remove entries from company_specialities for company_ids that don't exist in companies

In [11]:
company_specialities["speciality"] = company_specialities["speciality"].str.lower()
print(f"Number of rows in company_specialities before removing duplicate entries - {len(company_specialities)}")
print(f"Number of rows with duplicate data - {len([x for x in company_specialities.duplicated() if x==True])}")
company_specialities.drop_duplicates(inplace=True)
print(f"Number of rows in company_specialities after removing duplicate entries - {len(company_specialities)}")

print("company_specialities has multivalued column speciality.")
print(f"Number of rows before exploding multivalued rows - {len(company_specialities)}")
def split_specialities(specialities):
    parts = re.split(r',(?![^\(]*\))', specialities)
    return [part.strip() for part in parts]

company_specialities['speciality'] = company_specialities['speciality'].apply(split_specialities)
company_specialities = company_specialities.explode('speciality')
company_specialities.reset_index(drop=True, inplace=True)
print(f"Number of rows after exploding multivalued rows - {len(company_specialities)}")

print(f"Number of rows in company_specialities before removing duplicate entries - {len(company_specialities)}")
print(f"Number of rows with duplicate data - {len([x for x in company_specialities.duplicated() if x==True])}")
company_specialities.drop_duplicates(inplace=True)
print(f"Number of rows in company_specialities after removing duplicate entries - {len(company_specialities)}")

a = set(companies["company_id"])
b = set(company_specialities["company_id"])
print(f"Number of rows in company_specialities before enforcing foreign key constraint - {len(company_specialities)}")
print("These company_ids don't correspond to any row in companies. Removing them ...")
print(company_specialities.loc[company_specialities["company_id"].isin(b - a)])
company_specialities.drop(company_specialities[company_specialities["company_id"].isin(b - a)].index, inplace=True)
print(f"Number of rows in company_specialities after enforcing foreign key constraint - {len(company_specialities)}")
print(f"Number of rows in companies whose company don't have any company_specialities associated with it - {len(a - b)}")

Number of rows in company_specialities before removing duplicate entries - 128355
Number of rows with duplicate data - 85798
Number of rows in company_specialities after removing duplicate entries - 42557
company_specialities has multivalued column speciality.
Number of rows before exploding multivalued rows - 42557
Number of rows after exploding multivalued rows - 43555
Number of rows in company_specialities before removing duplicate entries - 43555
Number of rows with duplicate data - 46
Number of rows in company_specialities after removing duplicate entries - 43509
Number of rows in company_specialities before enforcing foreign key constraint - 43509
These company_ids don't correspond to any row in companies. Removing them ...
     company_id                    speciality
0      81149246     childrens music education
1      81149246     foundational music theory
2      81149246           child music lessons
3      81149246     social emotional learning
4      81149246  social emotio

## ER Diagram after enforcing constraints

```mermaid
erDiagram
    Company {
        int company_id PK
        string name
        string description
    }

    CompanyIndustry {
        int company_id FK,PK
        string industry
    }
    Company ||--o| CompanyIndustry : "part of"

    CompanySpeciality {
        int company_id FK,PK
        string speciality PK
    }
    Company ||--o{ CompanySpeciality : has

    EmployeeCount {
        int company_id FK
        float time_recorded
        int employee_count
        int follower_count
    }
    Company ||--o{ EmployeeCount : has
    
    JobPosting {
        int job_id PK
        int company_id FK
    }
    JobPosting }o..o| Company : by
    
    JobSkill {
        int job_id FK,PK
        string skill_abr PK
    }
    JobPosting ||--o{ JobSkill : has
    
    JobIndustry {
        int job_id FK,PK
        string industry_id PK
    }
    JobPosting ||--o{ JobIndustry : has
    
    Benefit {
        int job_id FK,PK
        string type PK
        bool inferred
    }
    JobPosting ||--o{ Benefit : has
```

**Strong Entities** - JobPosting, Company

**Weak Entities of JobPosting** - Benefit, JobIndustry, JobSkill

**Weak Entities of Company** - CompanyIndustry, CompanySpeciality, EmployeeCount

## Save to Staging Area (optimized for MySQL)

In [13]:
job_postings.to_csv('staging_data/job_postings.csv', index=False, sep=",", escapechar="\\", na_rep='NULL',
                    columns=['job_id', 'company_id', 'title', 'description', 'skills_desc', 'work_type', 'location', 'currency', 'remote_allowed', 'sponsored', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'compensation_type', 'formatted_work_type', 'formatted_experience_level', 'applies', 'views', 'original_listed_time', 'listed_time', 'expiry', 'closed_time', 'posting_domain', 'job_posting_url', 'application_url', 'application_type'])

benefits.to_csv('staging_data/benefits.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                    columns=['job_id', 'type', 'inferred'])

job_industries.to_csv('staging_data/job_industries.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                        columns=['job_id', 'industry_id'])

job_skills.to_csv('staging_data/job_skills.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                    columns=['job_id', 'skill_abr'])

companies.to_csv('staging_data/companies.csv', index=False, sep=",", escapechar="\\", na_rep='NULL',
                    columns=['company_id', 'name', 'description', 'company_size', 'address', 'city', 'state', 'country', 'zip_code', 'url'])

company_industries.to_csv('staging_data/company_industries.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                            columns=['company_id', 'industry'])

company_specialities.to_csv('staging_data/company_specialities.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                            columns=['company_id', 'speciality'])

employee_counts.to_csv('staging_data/employee_counts.csv', index=False, quoting=csv.QUOTE_NONNUMERIC,
                        columns=['company_id', 'time_recorded', 'employee_count', 'follower_count'])